In [1]:
import pandas as pd
pd.set_option('float_format', '{:f}'.format)

In [2]:
cdr = pd.read_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/dados_18.11/cdr/cdr_regiao_imediata_sjdr.csv', sep=';', header=None)
cdr.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,2013-03-21,11:51:30,0.370000,32,5B5F2C071D12AF13219DF5EBE05132AF,32,9FB3B96B6D5E16C9DD564AA3E84F1954,50141,nan,117,LOCAL,OI,OI,PRE PAGO,300000533
1,2013-03-21,21:03:30,0.130000,61,B3299B0E587D7275E3E4D530E9EECF50,98,6432F1DF21BA38368D9A165C739EEBB3,51601,nan,195,LONGA DISTANCIA,BRT,OI,PRE PAGO,300000210
2,2013-03-21,17:33:20,0.330000,61,85D5C50A6D882CA8E4BB00BCA3574417,61,0D8583F810B9720A8032BB939F12B3FF,51601,nan,117,LOCAL,BRT,BRT,PRE PAGO,300000210
3,2013-03-21,11:58:26,1.270000,61,6C10A9E9F325CAA3CCB7F9A0D6983D2A,61,B0C50ED1DEC9E06E4C64E7419DDC4B09,51601,nan,117,LOCAL,BRT,BRT,PRE PAGO,300000210
4,2013-03-21,16:06:59,0.170000,61,29122DCF4189448865B03F24C2DFEAE6,61,E0B097D86BF9C02A23011610EBAF47FD,51601,nan,77,LOCAL,BRT,GVT,PRE PAGO,300000210


In [3]:
cdr.rename(columns={0:'DATE', 1:'HOUR', 2:'DURATION', 3:'DDD_USER_FROM',
                    4:'USER_FROM', 5:'DDD_USER_TO', 6:'USER_TO', 7: 'ANTENNA'},inplace=True)
cdr = cdr.drop(8,1)

# Análise Exploratória

Parte 1: Análise sobre os usuários

In [4]:
#USER_FROM Distintos
len(cdr['USER_FROM'].unique())

445162

In [6]:
#USER_TO Distintos
len(cdr['USER_TO'].unique())

1198467

In [7]:
#Id's distintos em USER_FROM e USER_TO
len(pd.unique(cdr[['USER_FROM', 'USER_TO']].values.ravel('k')))

1303639

Parte 2: Análise sobre a duração das ligações

In [8]:
cdr['DURATION'].describe()

count   9293316.000000
mean          1.496004
std           8.311616
min           0.000000
25%           0.220000
50%           0.600000
75%           1.420000
max        5758.000000
Name: DURATION, dtype: float64

In [9]:
cdr['DURATION'].median()

0.6

# Filtros da base

Selecionando apenas registros onde a duração da ligação está entre 0.07 e 120 minutos

In [4]:
cdr_new = cdr[cdr['DURATION'].between(0.07,120)]

In [11]:
#Mesmo com o filtro de duração, manteve-se pouco mais de 99% dos registros da base original
len(cdr_new)/len(cdr)

0.9934378643747829

Filtrando a partir do número de ligações

In [5]:
#Selecionando os usuários com o número de ligações entre 3 e 500
cdr_filter_by_quantity_of_call = cdr_new.groupby('USER_FROM').size().reset_index().rename(columns={0:'QUANTITY_OF_CALLS'})

cdr_filter_by_quantity_of_call = cdr_filter_by_quantity_of_call.loc[(
                                (cdr_filter_by_quantity_of_call['QUANTITY_OF_CALLS'] >= 3) 
                                & (cdr_filter_by_quantity_of_call['QUANTITY_OF_CALLS'] <= 500))]

cdr_filter_by_quantity_of_call.head()

,USER_FROM,QUANTITY_OF_CALLS
1,000008B27F635FC0AFF9BAC21880B796,4
3,00002BC8FEAE1CB1EC0E1A8AF160B16E,12
4,00003F26576D1122848B6F3CEEC250E5,12
7,0000FFB82D6062A83F28FEE4D0BD5E4D,3
8,00014FE9BDE3EE54C61AB62700A1A2AF,14


In [6]:
#Selecionando os usuários com pelo menos 3 ligações em dias distintos
cdr_filter_by_quantity_of_days = cdr_new.groupby('USER_FROM').DATE.nunique().to_frame('QUANTITY_OF_DAYS').reset_index()
cdr_filter_by_quantity_of_days = cdr_filter_by_quantity_of_days.loc[(cdr_filter_by_quantity_of_days['QUANTITY_OF_DAYS'] >= 3)]
cdr_filter_by_quantity_of_days.head()

,USER_FROM,QUANTITY_OF_DAYS
1,000008B27F635FC0AFF9BAC21880B796,3
3,00002BC8FEAE1CB1EC0E1A8AF160B16E,6
4,00003F26576D1122848B6F3CEEC250E5,9
7,0000FFB82D6062A83F28FEE4D0BD5E4D,3
8,00014FE9BDE3EE54C61AB62700A1A2AF,5


In [7]:
#Interseção entre os dois dataframes (de ligações e dias com ligações)
df_filter_over_call = pd.merge(cdr_filter_by_quantity_of_call, cdr_filter_by_quantity_of_days,
                               how='inner',on=['USER_FROM'])

df_filter_over_call.head()

,USER_FROM,QUANTITY_OF_CALLS,QUANTITY_OF_DAYS
0,000008B27F635FC0AFF9BAC21880B796,4,3
1,00002BC8FEAE1CB1EC0E1A8AF160B16E,12,6
2,00003F26576D1122848B6F3CEEC250E5,12,9
3,0000FFB82D6062A83F28FEE4D0BD5E4D,3,3
4,00014FE9BDE3EE54C61AB62700A1A2AF,14,5


In [8]:
#Filtrando cdr_new: somente os USER_FROM que constam em df_filter_over_call continuaram na análise
cdr_after_filters_of_calls = cdr_new[(cdr_new['USER_FROM'].isin(df_filter_over_call['USER_FROM']))]
cdr_after_filters_of_calls.head()

,DATE,HOUR,DURATION,DDD_USER_FROM,USER_FROM,DDD_USER_TO,USER_TO,ANTENNA,9,10,11,12,13,14
0,2013-03-21,11:51:30,0.370000,32,5B5F2C071D12AF13219DF5EBE05132AF,32,9FB3B96B6D5E16C9DD564AA3E84F1954,50141,117,LOCAL,OI,OI,PRE PAGO,300000533
1,2013-03-21,21:03:30,0.130000,61,B3299B0E587D7275E3E4D530E9EECF50,98,6432F1DF21BA38368D9A165C739EEBB3,51601,195,LONGA DISTANCIA,BRT,OI,PRE PAGO,300000210
2,2013-03-21,17:33:20,0.330000,61,85D5C50A6D882CA8E4BB00BCA3574417,61,0D8583F810B9720A8032BB939F12B3FF,51601,117,LOCAL,BRT,BRT,PRE PAGO,300000210
3,2013-03-21,11:58:26,1.270000,61,6C10A9E9F325CAA3CCB7F9A0D6983D2A,61,B0C50ED1DEC9E06E4C64E7419DDC4B09,51601,117,LOCAL,BRT,BRT,PRE PAGO,300000210
4,2013-03-21,16:06:59,0.170000,61,29122DCF4189448865B03F24C2DFEAE6,61,E0B097D86BF9C02A23011610EBAF47FD,51601,77,LOCAL,BRT,GVT,PRE PAGO,300000210


Selecionando os registros onde as ligações ocorreram entre 19:00 e 06:00 ou tenham sido realizadas em qualquer horário do domingo

In [9]:
cdr_after_filters_of_calls['DATE'] = pd.to_datetime(cdr_after_filters_of_calls['DATE'])
cdr_after_filters_of_calls['DAY_OF_WEEK'] = cdr_after_filters_of_calls['DATE'].dt.day_name()

/home/gustavo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/gustavo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [18]:
cdr_after_filters_of_calls.sample(5)

,DATE,HOUR,DURATION,DDD_USER_FROM,USER_FROM,DDD_USER_TO,USER_TO,ANTENNA,9,10,11,12,13,14,DAY_OF_WEEK
905077,2013-03-23,08:03:06,0.850000,83,2442402450A0DCFC8C83B425B2CAC45E,83,0F4A4D6A1728115BC9C48E60D2612C3A,49592,117,LOCAL,OI,CLARO,PRE PAGO,300000167,Saturday
9163339,2013-04-19,09:32:39,2.970000,32,DFF818FBD798339F4AF9D7AB8AB6664F,32,480F34DC54936E3A4AB82D7859F2303F,23461,117,LOCAL,OI,OI,POS PAGO,300000563,Friday
6802849,2013-04-11,17:06:11,0.430000,32,42B16456ABB0FE1D5959C25F2DC41DF0,32,5F11685DED5117FB30E6CDB703611AB8,49592,77,LOCAL,OI,TNL,PRE PAGO,300000210,Thursday
8892088,2013-04-18,08:27:59,2.300000,32,62E6F6C2245A9F620DE5D5FE87C72B34,32,2B157766FCAE4C6C0CAF2763CEB806C9,49592,117,LOCAL,OI,OI,PRE PAGO,300000210,Thursday
5278875,2013-04-06,17:19:20,0.450000,32,77F2BCA04670C11F632807C02EC0C7EE,32,FC6416B790EDFDB52478BB2691CB27BC,23441,117,LOCAL,OI,OI,PRE PAGO,300000208,Saturday


In [10]:
#selecionando as ligações que ocorreram entre 19h e 6h ou no domingo em qualquer horário
calls_night_or_sunday = cdr_after_filters_of_calls[(cdr_after_filters_of_calls.HOUR.between('19:00:00','23:59:00')) 
                               | (cdr_after_filters_of_calls.HOUR.between('00:00:00','06:00:00')) 
                               | (cdr_after_filters_of_calls['DAY_OF_WEEK'] == 'Sunday')]

calls_night_or_sunday

,DATE,HOUR,DURATION,DDD_USER_FROM,USER_FROM,DDD_USER_TO,USER_TO,ANTENNA,9,10,11,12,13,14,DAY_OF_WEEK
1,2013-03-21,21:03:30,0.130000,61,B3299B0E587D7275E3E4D530E9EECF50,98,6432F1DF21BA38368D9A165C739EEBB3,51601,195,LONGA DISTANCIA,BRT,OI,PRE PAGO,300000210,Thursday
6,2013-03-21,20:30:29,7.020000,61,BA4DEA1F710C1216F74F1255150C49D6,61,4F6DCBA36292B8487743E13319E67DF6,51601,77,LOCAL,BRT,GVT,PRE PAGO,300000210,Thursday
9,2013-03-21,01:10:47,0.830000,32,1AC8490BA99709F652B4A5C3525D3C82,32,B6BB8A38B286A41978E732BDE0249DB5,50141,117,LOCAL,OI,OI,POS PAGO,300000188,Thursday
10,2013-03-21,21:11:55,6.870000,32,05AC98D91ED3C2821F53519B19CCBC58,32,A3ACD65D308B86F4487DB35A5F2E50EB,50141,117,LOCAL,OI,OI,PRE PAGO,300000533,Thursday
13,2013-03-21,20:29:21,0.130000,61,BA4DEA1F710C1216F74F1255150C49D6,21,A49BA005920B8BA24BEE40F455B69B9B,51601,117,LOCAL,BRT,OI,PRE PAGO,300000210,Thursday
15,2013-03-21,00:09:27,0.130000,61,023CF68D76C38796F6C3B138F27AC657,21,A49BA005920B8BA24BEE40F455B69B9B,51601,117,LOCAL,BRT,OI,PRE PAGO,300000210,Thursday
18,2013-03-21,22:36:30,1.220000,61,B046BF714E16102B098297E89639BA30,61,A943CD0FB95D587FF5D10F61D643AD73,51601,117,LOCAL,BRT,BRT,PRE PAGO,300000210,Thursday
24,2013-03-21,19:22:23,0.170000,32,22952FD19F943E892FE0D2547FC58CB2,32,217FFDF4EABCC0C22984D2618BD5AEF8,50141,117,LOCAL,OI,AMC/TMC,PRE PAGO,300000210,Thursday
30,2013-03-21,22:05:54,12.470000,61,E42DC46C0AA235698C6D45E88B9A032A,61,063BF2D37E4804EF78F98D41CA3596F0,51601,117,LOCAL,BRT,BRT,PRE PAGO,300000210,Thursday
32,2013-03-21,23:45:49,17.770000,32,DCB4C7B49814690E47092C87C0242BFE,32,544A1CD1C371AE2D7BB8A51AAE9673E2,49592,117,LOCAL,OI,OI,POS PAGO,300000292,Thursday


In [11]:
calls_night_or_sunday.to_csv('/home/gustavo/Desktop/Mestrado/mestrado_dados/dados_18.11/cdr_after_filtes.csv')